In [7]:
import numpy as np
from scipy import linalg

def von_neumann_entropy(alpha: np.ndarray) -> np.ndarray:
    r""" Computes the von neumann entropy of a the partial density matrix
    of the first subsystem of the total system described by state `alpha`.
    Note that this function does not support more than a two-mode system for now.

    Args:
        alpha (np.ndarray): The coefficients of the state of the total system expressed in the Fock basis.
    Returns:
        (np.ndarray): The von neumann entropy of the first subsystem.
    """

    # Let us compute the partial density matrix of the first
    # subsystem, expressed in the Fock basis
    rho = np.einsum('ml,nl->nm', alpha.conjugate(), alpha)

    # We finally compute the von Neumann entropy
    entropy = -np.trace(rho @ linalg.logm(rho))

    return entropy

In [2]:
alpha = np.array(
    [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ]
)


In [21]:
type(von_neumann_entropy(alpha=alpha).item())

float

In [9]:
rho = np.einsum('ml,nl->nm', alpha.conjugate(), alpha)

In [9]:
import itertools
dim1 = 2
dim2 = 4

input = ((i, j) for i, j in itertools.product(range(dim1), range(dim2)))
print(list(input))

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3)]


In [38]:
alpha = np.zeros(shape=(9, 70, 5, 5))

In [23]:
a = np.array(results).reshape((2, 3, 2, 2))

In [27]:
a[1, 2]

array([[ 1,  3],
       [-1,  2]])

In [5]:
import numpy as np
import string


In [8]:
a = np.array([[1, 2], [3, 4]])
n = 3

In [33]:
einsum_rule = ','.join([string.ascii_lowercase[n+1] + string.ascii_lowercase[n+2] + string.ascii_lowercase[i: i+2] for i in range(n)]) + '->' + string.ascii_lowercase[0] +string.ascii_lowercase[n]
np.einsum(einsum_rule, *(a for _ in range(n)))

array([[ 37,  54],
       [ 81, 118]])

In [37]:
einsum_rule = ','.join([string.ascii_lowercase[n+1] + string.ascii_lowercase[n+2] + string.ascii_lowercase[i: i+2] for i in range(n)]) + '->' + string.ascii_lowercase[n+1] + string.ascii_lowercase[n+2] + string.ascii_lowercase[0] + string.ascii_lowercase[n]
einsum_rule

'efab,efbc,efcd->efad'

In [39]:
from src.utils import renyi_entropy

renyi_entropy(alpha)

ModuleNotFoundError: No module named 'strawberryfields'